In [1]:
import os
import sys
import json
from pathlib import Path
sys.path.append(os.path.abspath('../src'))

# ----------- local imports ----------- 
from utils import show_images
from constants import DATA_DIR

In [2]:
from deepface import DeepFace

2025-03-12 02:28:51.068813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741739331.107720   27210 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741739331.119582   27210 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 02:28:51.165033: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
??DeepFace

Type:        module
String form: <module 'deepface.DeepFace' from '/home/aliaagheis/miniconda3/envs/fast_env/lib/python3.12/site-packages/deepface/DeepFace.py'>
File:        ~/miniconda3/envs/fast_env/lib/python3.12/site-packages/deepface/DeepFace.py
Source:     
# common dependencies
import os
import warnings
import logging
from typing import Any, Dict, List, Union, Optional

# this has to be set before importing tensorflow
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# pylint: disable=wrong-import-position

# 3rd party dependencies
import numpy as np
import pandas as pd
import tensorflow as tf

# package dependencies
from deepface.commons import package_utils, folder_utils
from deepface.commons.logger import Logger
from deepface.modules import (
    modeling,
    representation,
    verification,
    recognition,
    demography,
    detection,
    streaming,
    preprocessing,
)
from deepface import __version__

logger = Logger()

# -----------------------------------
# configurations fo

In [3]:
dfs = DeepFace.represent(
  img_path = "pp.jpg", 
  # db_path = DATA_DIR / "face_identification/train", 
  model_name = 'Facenet512',
  # detector_backend='opencv'  # Try 'mtcnn', 'retinaface', etc.
  enforce_detection=False
)

2025-03-12 02:29:11.414608: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1741739351.414876   27210 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
E0000 00:00:1741739356.838312   27210 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2025-03-12 02:29:16.839672: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at conv_ops_impl.h:1204 : INVALID_AR

InvalidArgumentError: Exception encountered when calling layer 'Conv2d_1a_3x3' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D] name: 

Call arguments received by layer 'Conv2d_1a_3x3' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 160, 160, 3), dtype=float32)

In [8]:
from PIL import Image

In [30]:
img_sample = Image.open(DATA_DIR / "face_identification/train/person_0/0.jpg")

img_sample = img_sample.resize((160, 160), Image.LANCZOS)  # LANCZOS for high-quality downsampling

img_sample.save("pp.jpg")

In [31]:
# train_path = DATA_DIR / "face_identification/test/"

# imgs_paths = train_path.rglob("*.jpg")

# [Image.open(img).size for img in imgs_paths]

In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'2.18.0'